## Import general Python libraries, LIWC

In [2]:
import os
# !pip install pandas
import pandas as pd
import numpy as np
%pip install scikit-learn
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
%pip install matplotlib
import matplotlib.pyplot as plt
%pip install nltk
import nltk 
from nltk.corpus import stopwords
import collections
%pip install liwc
import liwc
from collections import Counter

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Import nltk libraries and download packages 

In [3]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag, ne_chunk
import string
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.sentiment import SentimentIntensityAnalyzer

In [4]:
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('brown')
nltk.download('vader_lexicon')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/danakurniawan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danakurniawan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/danakurniawan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danakurniawan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/danakurniawan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/danakurniawan/nltk_data...
[nltk_data]   Package maxent_ne_chunker i

True

## Text features extraction

In [12]:
# df_train = pd.read_csv("/Users/danakurniawan/Documents/Columbia/COMS_6706/hw2/train.csv")
# df_val = pd.read_csv("/Users/danakurniawan/Documents/Columbia/COMS_6706/hw2/valid.csv") 
df_test = pd.read_csv("/Users/danakurniawan/Documents/Columbia/COMS_6706/hw2/test.csv")

#check that shape is still the same
print(f"Total rows in dataset: {len(df_test)}")
print(f"Columns: {df_test.columns.tolist()}")
print(f"\nFirst few transcript values:")
print(df_test['transcript'].head(10))

def extract_nltk_features(text):

    """Extract NLTK features from train.csv for text-based set"""
    if pd.isna(text) or text == 'SIL':
        return {
            #tokenization
            'word_count': 0,
            'sentence_count': 0,
            'avg_sentence_length': 0,

            #parts of speech tags and categories
            'noun_count': 0,
            'proper_noun_count': 0,
            'verb_count': 0,
            'modal_verb_count': 0,
            'adj_count': 0,
            'adv_count': 0,
            'pronoun_count': 0,
            'determiner_count': 0,
            'preposition_count': 0,
            'conjunction_count': 0,
            'interjection_count': 0,
            'particle_count': 0,
            'pos_tag_diversity': 0,

            # Named Entity Recognition
            'named_entity_count': 0,
            'person_entity_count': 0,
            'organization_entity_count': 0,
            'location_entity_count': 0,
            
            # Stopwords
            'stopword_count': 0,
            'stopword_ratio': 0,
            
            # N-gram features
            'unique_bigrams': 0,
            'unique_trigrams': 0,
            'bigram_diversity': 0,
            
            # Sentiment (VADER from NLTK)
            'sentiment_positive': 0,
            'sentiment_negative': 0,
            'sentiment_neutral': 0,
            'sentiment_compound': 0,
        }
    
    #normalize the text and create the feature set
    text = str(text)
    tokens = word_tokenize(text.lower())
    tokens_original = word_tokenize(text)
    sentences = sent_tokenize(text)
    
    features['word_count'] = len(tokens)
    features['sentence_count'] = len(sentences)
    features['avg_sentence_length'] = len(tokens) / len(sentences) if sentences else 0
    
    #pos features
    pos_tags = pos_tag(tokens_original)
    pos_tag_list = [tag for word, tag in pos_tags]

    #count different POS categories
    features['noun_count'] = sum(1 for tag in pos_tag_list if tag.startswith('NN'))
    features['proper_noun_count'] = sum(1 for tag in pos_tag_list if tag in ['NNP', 'NNPS'])
    features['verb_count'] = sum(1 for tag in pos_tag_list if tag.startswith('VB'))
    features['modal_verb_count'] = sum(1 for tag in pos_tag_list if tag == 'MD')
    features['adj_count'] = sum(1 for tag in pos_tag_list if tag.startswith('JJ'))
    features['adv_count'] = sum(1 for tag in pos_tag_list if tag.startswith('RB'))
    features['pronoun_count'] = sum(1 for tag in pos_tag_list if tag.startswith('PRP'))
    features['determiner_count'] = sum(1 for tag in pos_tag_list if tag == 'DT')
    features['preposition_count'] = sum(1 for tag in pos_tag_list if tag == 'IN')
    features['conjunction_count'] = sum(1 for tag in pos_tag_list if tag in ['CC', 'IN'])
    features['interjection_count'] = sum(1 for tag in pos_tag_list if tag == 'UH')
    features['particle_count'] = sum(1 for tag in pos_tag_list if tag == 'RP')

    # POS tag diversity
    unique_pos_tags = len(set(pos_tag_list))
    features['pos_tag_diversity'] = unique_pos_tags / len(pos_tag_list) if pos_tag_list else 0
    
    #named entity recognition (NER)
    try:
        tree = ne_chunk(pos_tags, binary=False)
        entities = []
        person_count = 0
        org_count = 0
        location_count = 0
        
        for subtree in tree:
            if hasattr(subtree, 'label'):
                entity_label = subtree.label()
                entities.append(entity_label)
                if entity_label == 'PERSON':
                    person_count += 1
                elif entity_label == 'ORGANIZATION':
                    org_count += 1
                elif entity_label in ['GPE', 'LOCATION']:
                    location_count += 1
        
        features['named_entity_count'] = len(entities)
        features['person_entity_count'] = person_count
        features['organization_entity_count'] = org_count
        features['location_entity_count'] = location_count
    except:
        features['named_entity_count'] = 0
        features['person_entity_count'] = 0
        features['organization_entity_count'] = 0
        features['location_entity_count'] = 0
    
    #get stopwords
    stop_words = set(stopwords.words('english'))
    stopword_count = sum(1 for word in tokens if word in stop_words)
    features['stopword_count'] = stopword_count
    features['stopword_ratio'] = stopword_count / len(tokens) if tokens else 0
    
    #get n_grams 
    if len(tokens) >= 2:
        bigrams_list = list(ngrams(tokens, 2))
        features['unique_bigrams'] = len(set(bigrams_list))
        features['bigram_diversity'] = len(set(bigrams_list)) / len(bigrams_list) if bigrams_list else 0
    else:
        features['unique_bigrams'] = 0
        features['bigram_diversity'] = 0
    
    if len(tokens) >= 3:
        trigrams_list = list(ngrams(tokens, 3))
        features['unique_trigrams'] = len(set(trigrams_list))
    else:
        features['unique_trigrams'] = 0

    #sentiment analysis
    try:
        sentiment_scores = sia.polarity_scores(text)
        features['sentiment_positive'] = sentiment_scores['pos']
        features['sentiment_negative'] = sentiment_scores['neg']
        features['sentiment_neutral'] = sentiment_scores['neu']
        features['sentiment_compound'] = sentiment_scores['compound']
    except:
        features['sentiment_positive'] = 0
        features['sentiment_negative'] = 0
        features['sentiment_neutral'] = 0
        features['sentiment_compound'] = 0

    return features

#extract NLTK features from transcript
print("\n extracting features from every row in the set...") 
nltk_features_list = []

for idx, text in enumerate(df_test['transcript']):
    if idx % 1000 == 0:
        print(f"Processing row {idx}/{len(df_test)}...")
    features = extract_nltk_features(text)
    nltk_features_list.append(features)

print(f"Finished processing all {len(nltk_features_list)} rows")

# Convert to DataFrame
nltk_features_df = pd.DataFrame(nltk_features_list)

# Keep only the original columns (all columns from loaded CSV)
# Since we want to drop dialog_id and speaker, let's explicitly select what we want
original_columns = [col for col in df_test.columns if col not in ['dialog_id', 'speaker']]
df_test_selected = df_test[original_columns].copy()

# Combine with NLTK features
df_test_with_nltk = pd.concat([df_test_selected.reset_index(drop=True), 
                             nltk_features_df.reset_index(drop=True)], axis=1)

# Save the enhanced dataset
df_test_with_nltk.to_csv('text_features_test.csv', index=False)

print(f"\n{'='*60}")
print(f"processing done for NLTK features")
print(f"{'='*60}")
print(f"Original shape: {df_test.shape}")
print(f"New shape (first 2 columns dropped): {df_test_with_nltk.shape}")
print(f"Rows processed: {len(nltk_features_list)}")
print(f"\nNLTK features extracted ({len(nltk_features_df.columns)} total):")
print(list(nltk_features_df.columns)) 

print(f"\nFirst few rows with key features:")
# Check what columns actually exist
print("\nColumns in final dataset:")
print(df_test_with_nltk.columns.tolist())

print("nltk_features_df shape:", nltk_features_df.shape)
print("nltk_features_df columns:", nltk_features_df.columns.tolist())

# Print first few rows of any columns that exist
print("\nFirst few rows:")
print(df_test_with_nltk.head(15))

print(f"\n check for any missing values in extracted features:")
print(nltk_features_df.isnull().sum().sum(), "missing values found")

print(f"\n saved to: text_features_test.csv")

Total rows in dataset: 23540
Columns: ['dialog_id', 'speaker', 'transcript', 'da_tag', 'start_time', 'end_time', 'function', 'pronoun', 'ppron', 'i', 'we', 'you', 'shehe', 'they', 'ipron', 'article', 'prep', 'auxverb', 'adverb', 'conj', 'negate', 'verb', 'adj', 'compare', 'interrog', 'number', 'quant', 'affect', 'posemo', 'negemo', 'anx', 'anger', 'sad', 'social', 'family', 'friend', 'female', 'male', 'cogproc', 'insight', 'cause', 'discrep', 'tentat', 'certain', 'differ', 'percept', 'see', 'hear', 'feel', 'bio', 'body', 'health', 'sexual', 'ingest', 'drives', 'affiliation', 'achieve', 'power', 'reward', 'risk', 'focuspast', 'focuspresent', 'focusfuture', 'relativ', 'motion', 'space', 'time', 'work', 'leisure', 'home', 'money', 'relig', 'death', 'informal', 'swear', 'netspeak', 'assent', 'nonflu', 'filler']

First few transcript values:
0                                   SIL
1                                   SIL
2            uh do you have a pet randy
3                              

# Speech features extraction  
## (also available in speech_features.py due to running with asyncio)

In [13]:
import os
import pandas as pd
from tqdm import tqdm
import asyncio
from typing import Optional
import numpy as np
import librosa
from librosa.feature import mfcc, rms, zero_crossing_rate

In [ ]:
class CustomProcessor():
    def __init__(self):
        # self.processor_rosa = librosa
        # self.processor_praat = None
        # self.sr = sampling_rate
        pass

    def __call__(self, y, custom_sr):  

        # print("IN call", end="")
        output = np.zeros(45) #initialize for all features
        # print(y.shape)
        # print(rms(y=y, frame_length=y.size))

        #handle short utterances by restricting?
        frame_len = len(y) if len(y) > 0 else 512
        n_fft_size = len(y) if len(y) > 0 else 512

        #rms for amplitude
        rms_values = rms(y=y, frame_length=y.size)[0]
        output[0] = np.mean(rms_values) #mean
        output[1] = np.std(rms_values) #std dev 
        output[2] = np.max(rms_values) #max
        output[3] = np.min(rms_values) #min

        #zero crossing rate for crossing above y axis used to separate eg. consonants, vowels 
        zcr = zero_crossing_rate(y, frame_length=frame_len)[0]
        output[4] = np.mean(zcr)
        output[5] = np.std(zcr)

        # Decided not to factor in spectral centroid - too expensive to compute (>20 hours)
        #spectral_centroid
        # spectral_centroid_val = np.squeeze(spectral_centroid(y=y, n_fft=n_fft_size)) # Use squeeze instead of indexing the first one
        # print(spectral_centroid(y=y, n_fft=n_fft_size))
        # # This is where function termination happened previously, because this always caused an error (because it was calling mean() on a function, not an array)
        # output[6] = np.mean(spectral_centroid_val)
        # output[7] = np.std(spectral_centroid_val)

        # #mfcc - mel frequency values
        # mfcc_features = mfcc(y=y, sr=custom_sr, n_mfcc=13, n_fft=n_fft_size)
        # for i in range(13):
        #     output[8 + i] = np.mean(mfcc_features[i]) # MFCC means at [indices] 8-20
        #     output[8 + 13 + i] = np.std(mfcc_features[i]) #MFCC stds at [indices] 21-33
    
        #pitch and magnitude 
        pitches, magnitudes = librosa.piptrack(y=y, n_fft=n_fft_size)
        
        # Extract pitch values where magnitude is high
        pitch_vals = []
        for t in range(pitches.shape[1]):
            index = magnitudes[:, t].argmax()
            pitch = pitches[index, t]
            if pitch > 0:
                pitch_vals.append(pitch)
        
        if len(pitch_vals) > 0:
            output[34] = np.mean(pitch_vals)
            output[35] = np.std(pitch_vals)
            output[36] = np.max(pitch_vals)
            output[37] = np.min(pitch_vals)
            output[38] = np.max(pitch_vals) - np.min(pitch_vals)  # pitch range
        
        # duration
        output[39] = len(y) / 22050  # sr = 22050 Duration in seconds
        output[40] = librosa.feature.tempo(y=y, sr=custom_sr)[0] if len(y) > custom_sr else 0
        # print(" END call")
             
        return output

    # #in case of errors output zeros
    # def _return_zeros(self):
    #     """Return zero features for failed extractions (kept for compatibility)"""
    #     features = {
    #         'rms_mean': 0, 'rms_std': 0, 'rms_max': 0, 'rms_min': 0,
    #         'zcr_mean': 0, 'zcr_std': 0,
    #         'spectral_centroid_mean': 0, 'spectral_centroid_std': 0,
    #         'pitch_mean': 0, 'pitch_std': 0, 'pitch_max': 0, 'pitch_min': 0, 'pitch_range': 0,
    #         'duration': 0, 'tempo': 0, 
    #     }
    #     # Add MFCC zeros
    #     for i in range(13):
    #         features[f'mfcc_{i}_mean'] = 0
    #         features[f'mfcc_{i}_std'] = 0
    #     return features   


def process_row(
    row:pd.Series,
    processor:CustomProcessor,
    base_path: str, #add base path
    progress_callback:Optional[callable] = None,
    ):
    
    # Get the information specifying where to calculate the speech information
    dialog_id = row["dialog_id"]
    speaker = row["speaker"]
    start_time = row["start_time"]
    end_time = row["end_time"]
    duration = end_time - start_time

    file_name = f"{dialog_id}_{speaker}.wav"
    file_path = os.path.join(base_path, "wav", file_name)
    
    #load audio segment 
    print(f"Loading sample {dialog_id} beginning at time {start_time}")
    # y, sample_rate = librosa.load(file_path, sr=None, offset=start_time, duration=duration)
    y, sample_rate = librosa.load(file_path, offset=start_time, duration=duration)

    # # ret = processor(y, sr=sample_rate)  # Didn't define the processor to take sr, probably not needed either way
    ret = processor(y, sample_rate)
    print(f"Finished processing sample at time {start_time}")

    if progress_callback:
        progress_callback()

    return ret


num_cores = 20
sema = asyncio.Semaphore(num_cores)

async def process_row_v2(
    row:pd.Series,
    processor:CustomProcessor,
    base_path: str, #add base path
    progress_callback:Optional[callable] = None,
    ):

    async with sema:
    
        # Get the information specifying where to calculate the speech information
        dialog_id = row["dialog_id"]
        speaker = row["speaker"]
        start_time = row["start_time"]
        end_time = row["end_time"]
        duration = end_time - start_time

        file_name = f"{dialog_id}_{speaker}.wav"
        file_path = os.path.join(base_path, "wav", file_name)
        
        #load audio segment
        # sample_rate = 22050 ?? 
        # print(f"Loading sample {dialog_id} beginning at time {start_time}")
        # y, sample_rate = librosa.load(file_path, sr=None, offset=start_time, duration=duration)
        y, sample_rate = librosa.load(file_path, offset=start_time, duration=duration)

        # # ret = processor(y, sr=sample_rate)  # Didn't define the processor to take sr, probably not needed either way
        ret = processor(y, sample_rate)
        # print(f"Finished processing sample at time {start_time}")

        if progress_callback:
            progress_callback()

        return ret

async def process_rows(
    rows:pd.DataFrame,
    processor:CustomProcessor,
    base_path: str, #add path
    ) -> list[np.ndarray]:

    prog = tqdm(total=rows.shape[0])
    # num_cores = 1
    # sema = asyncio.Semaphore(num_cores)
    # async with sema:

    # results = await asyncio.gather(
    #     *[
    #         # asyncio.to_thread(process_row, row, processor, base_path, prog.update) for _, row in rows.iterrows()
    #         asyncio.to_thread(process_row, row, processor, base_path, prog.update) for _, row in rows.iterrows()
    #         # asyncio.to_thread(process_wav, row, processor, prog.update) for _, row in rows.iterrows()
    #     ]
    # )

    
    # num_cores = 16
    # sema = asyncio.Semaphore(num_cores)
    # async with sema:

    tasks = [
        process_row_v2(row, processor, base_path, prog.update)
        for _, row in rows.iterrows()
    ]

    results = await asyncio.gather(*tasks)

    prog.close()
    return results
    # prog.close() 

if __name__ == "__main__":
    processor = CustomProcessor()
    # base_path = "/Users/danakurniawan/Documents/Columbia/COMS_6706/hw2"
    base_path = "."

    # metadata = pd.read_csv(os.path.join(base_path, "train.csv")) #Create speech train set
    # metadata = pd.read_csv(os.path.join(base_path, "valid.csv")) #Create speech validation set
    metadata = pd.read_csv(os.path.join(base_path, "test.csv")) #Create speech test set
    print("finished loading metadata")

    # test on a single row 
    print("\n testing on the first row")
    test_row = metadata.iloc[1]
    test_process_on_row = process_row(test_row, processor, base_path)

    # # features = asyncio.run(process_rows(trunc_metadata, processor, base_path))

    # # Try running with semaphore to limit number of threads
    # features = asyncio.run(process_rows(trunc_metadata, processor, base_path))

    # extract features asynchronously across rows in a version like this
    features = asyncio.run(process_rows(metadata, processor, base_path)) #features is a list of arrays
    print("finished extracting speech features")

    # # TODO: save, or run with -i to not lose the computation at the end
    saved_features = np.array(features)
    np.save("speech_features_test.npy", saved_features)

## convert to .csv from .npy

In [ ]:
import numpy as np
import pandas as pd

# Load the .npy file
input_file = 'speech_features_train.npy'
data = np.load(input_file)
df = pd.DataFrame(data)

# save training to CSV
output_file = 'speech_features_train.csv'
df.to_csv(output_file, index=False)
print(f"\saved to {output_file}")
print(f"CSV file contains {df.shape[0]} rows and {df.shape[1]} columns")

#validation
input_file_valid = 'speech_features_valid.npy'
data_validation = np.load(input_file_valid)
df_validation = pd.DataFrame(data_validation)
output_file_validation = 'speech_features_valid.csv'
df_validation.to_csv(output_file_validation, index=False)
print(f"\saved to {output_file_validation}")
print(f"CSV file contains {df_validation.shape[0]} rows and {df_validation.shape[1]} columns")

#test
input_file_test = 'speech_features_test.npy'
data_test = np.load(input_file_test)
df_test = pd.DataFrame(data_test)
output_file_test = 'speech_features_test.csv'
df_test.to_csv(output_file_test, index=False)
print(f"\saved to {output_file_test}")
print(f"CSV file contains {df_test.shape[0]} rows and {df_test.shape[1]} columns")

# Print information about the data
print(f"Loaded array shape: {data.shape}")
print(f"Data type: {data.dtype}")
print(f"\nFirst few rows:")
print(data[:5])